# Hedge Your Bets - AI-Powered Sports Betting Analysis

## Model Training Pipeline

This notebook implements the complete ML pipeline for predicting player prop outcomes and evaluating betting value.

### Approach Overview:
1. **Data Preprocessing**: Load and clean player/team weekly stats
2. **Feature Engineering**: Create rolling averages, team context, game context
3. **Model Training**: Per-position LightGBM quantile models
4. **Evaluation**: Betting-relevant metrics and profit simulation

## Important note for inference:

Will need to collect player, player team, opponent team, and market from user. Then the backend will pulls the latest season data and constructs the exact features the model expects

In [21]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML libraries
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"LightGBM version: {lgb.__version__}")


Libraries imported successfully!
Pandas version: 2.2.2
LightGBM version: 4.6.0


## 1. Data Loading and Exploration

First, let's load and explore our datasets to understand the structure and data quality.


In [2]:
# Set up data paths
data_dir = Path("../datasets")
player_weekly_dir = data_dir / "player_weekly_stats"
team_weekly_dir = data_dir / "team_season_stats"
players_file = data_dir / "players.csv"

print("Data directory structure:")
print(f"Player weekly stats: {player_weekly_dir}")
print(f"Team weekly stats: {team_weekly_dir}")
print(f"Players metadata: {players_file}")

# Check what files we have
player_files = list(player_weekly_dir.glob("*.csv"))
team_files = list(team_weekly_dir.glob("*.csv"))

print(f"\nFound {len(player_files)} player weekly stat files")
print(f"Found {len(team_files)} team weekly stat files")
print(f"Years covered: {sorted([f.stem.split('_')[-1] for f in player_files])}")


Data directory structure:
Player weekly stats: ..\datasets\player_weekly_stats
Team weekly stats: ..\datasets\team_season_stats
Players metadata: ..\datasets\players.csv

Found 26 player weekly stat files
Found 26 team weekly stat files
Years covered: ['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [3]:
# Load and examine a sample of player data
sample_year = "2024"
player_sample = pd.read_csv(player_weekly_dir / f"stats_player_week_{sample_year}.csv")

print(f"Player data sample ({sample_year}):")
print(f"Shape: {player_sample.shape}")
print(f"Columns: {list(player_sample.columns)}")
print(f"\nFirst few rows:")
player_sample.head()


Player data sample (2024):
Shape: (18981, 114)
Columns: ['player_id', 'player_name', 'player_display_name', 'position', 'position_group', 'headshot_url', 'season', 'week', 'season_type', 'team', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_cpoe', 'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share', 'wopr', 'special_teams_tds', 'def_tackles_solo', 'def_tackles_wi

,player_id,player_name,player_display_name,position,position_group,headshot_url,season,week,season_type,team,...,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,2024,1,REG,NYJ,...,0,0,NaN,0,0,0,0,0,8.58,8.58
1,00-0023853,M.Prater,Matt Prater,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2024,1,REG,ARI,...,0,0,1.0,0,0,0,0,0,0.00,0.00
2,00-0025565,N.Folk,Nick Folk,K,SPEC,https://static.www.nfl.com/image/upload/f_auto...,2024,1,REG,TEN,...,0,0,1.0,0,0,0,0,0,0.00,0.00
3,00-0026190,C.Campbell,Calais Campbell,DE,DL,https://static.www.nfl.com/image/upload/f_auto...,2024,1,REG,MIA,...,0,0,NaN,0,0,0,0,0,0.00,0.00
4,00-0026498,M.Stafford,Matthew Stafford,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,2024,1,REG,LA,...,0,0,NaN,0,0,0,0,0,14.68,14.68


In [4]:
# Load and examine team data
team_sample = pd.read_csv(team_weekly_dir / f"stats_team_week_{sample_year}.csv")

print(f"Team data sample ({sample_year}):")
print(f"Shape: {team_sample.shape}")
print(f"Columns: {list(team_sample.columns)}")
print(f"\nFirst few rows:")
team_sample.head()


Team data sample (2024):
Shape: (570, 102)
Columns: ['season', 'week', 'team', 'season_type', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_cpoe', 'passing_2pt_conversions', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'special_teams_tds', 'def_tackles_solo', 'def_tackles_with_assist', 'def_tackle_assists', 'def_tackles_for_loss', 'def_tackles_for_loss_yards', 'def_fumbles_forced', 'def_sacks', 'def_sack_yards', 'def_qb_hits', 'def

,season,week,team,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,...,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance
0,2024,1,ARI,REG,BUF,21,31,162,1,0,...,2,2,0,0,1.0,0,0,0,0,0
1,2024,1,ATL,REG,PIT,16,26,155,1,2,...,1,1,0,0,1.0,0,0,0,0,0
2,2024,1,BAL,REG,KC,26,41,273,1,0,...,2,2,0,0,1.0,0,0,0,0,0
3,2024,1,BUF,REG,ARI,18,23,232,2,0,...,4,4,0,0,1.0,0,0,0,0,0
4,2024,1,CAR,REG,NO,13,31,161,0,2,...,1,1,0,0,1.0,0,0,0,0,0


In [5]:
# Load players metadata
players_meta = pd.read_csv(players_file)

print(f"Players metadata:")
print(f"Shape: {players_meta.shape}")
print(f"Key columns: {['gsis_id', 'display_name', 'position', 'position_group']}")
print(f"\nPosition distribution:")
print(players_meta['position_group'].value_counts())
print(f"\nSample players:")
players_meta[['gsis_id', 'display_name', 'position', 'position_group']].head(10)


Players metadata:
Shape: (24302, 39)
Key columns: ['gsis_id', 'display_name', 'position', 'position_group']

Position distribution:
position_group
DB      4357
OL      4002
DL      3421
LB      3357
WR      3169
RB      2625
TE      1545
QB       990
SPEC     836
Name: count, dtype: int64

Sample players:


,gsis_id,display_name,position,position_group
0,00-0028830,Isaako Aaitui,NT,DL
1,00-0038389,Israel Abanikanda,RB,RB
2,00-0024644,Jon Abbate,LB,LB
3,ABB498348,Vince Abbott,K,SPEC
4,00-0031021,Jared Abbrederis,WR,WR
5,00-0032860,Mehdi Abdesmad,DE,DL
6,00-0028564,Isa Abdul-Quddus,S,DB
7,00-0032104,Ameer Abdullah,RB,RB
8,00-0023663,Hamza Abdullah,DB,DB
9,00-0025940,Husain Abdullah,FS,DB


## 2. Data Preprocessing Pipeline

Now let's build a comprehensive data preprocessing pipeline that:
1. Loads all historical data
2. Merges player and team information
3. Creates temporal features
4. Handles missing values and data quality issues


In [6]:
class DataPreprocessor:
    """Comprehensive data preprocessing pipeline for sports betting analysis."""
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.player_weekly_dir = self.data_dir / "player_weekly_stats"
        self.team_weekly_dir = self.data_dir / "team_season_stats"
        self.players_file = self.data_dir / "players.csv"
        
        # Load players metadata once
        self.players_meta = pd.read_csv(self.players_file)
        
        # Define key stats for each position
        self.position_stats = {
            'QB': ['passing_yards', 'passing_tds', 'passing_interceptions', 'completions', 'attempts'],
            'RB': ['rushing_yards', 'rushing_tds', 'carries', 'receptions', 'receiving_yards'],
            'WR': ['receiving_yards', 'receptions', 'receiving_tds', 'targets'],
            'TE': ['receiving_yards', 'receptions', 'receiving_tds', 'targets'],
            'K': ['fg_made', 'fg_att', 'pat_made', 'pat_att']
        }
        
    def load_all_player_data(self, start_year=1999, end_year=2024):
        """Load all player weekly data for specified years."""
        all_data = []
        
        for year in range(start_year, end_year + 1):
            file_path = self.player_weekly_dir / f"stats_player_week_{year}.csv"
            if file_path.exists():
                df = pd.read_csv(file_path)
                all_data.append(df)
                print(f"Loaded {year}: {df.shape[0]} records")
            else:
                print(f"Warning: No data found for {year}")
        
        if all_data:
            combined = pd.concat(all_data, ignore_index=True)
            print(f"Total player records loaded: {combined.shape[0]}")
            return combined
        else:
            return pd.DataFrame()
    
    def load_all_team_data(self, start_year=1999, end_year=2024):
        """Load all team weekly data for specified years."""
        all_data = []
        
        for year in range(start_year, end_year + 1):
            file_path = self.team_weekly_dir / f"stats_team_week_{year}.csv"
            if file_path.exists():
                df = pd.read_csv(file_path)
                all_data.append(df)
                print(f"Loaded team data {year}: {df.shape[0]} records")
            else:
                print(f"Warning: No team data found for {year}")
        
        if all_data:
            combined = pd.concat(all_data, ignore_index=True)
            print(f"Total team records loaded: {combined.shape[0]}")
            return combined
        else:
            return pd.DataFrame()

# Initialize preprocessor
preprocessor = DataPreprocessor("../datasets")
print("DataPreprocessor initialized successfully!")


DataPreprocessor initialized successfully!


In [7]:
# Load a subset of data for initial testing (2020-2024)
print("Loading recent data for testing...")
player_data = preprocessor.load_all_player_data(start_year=2020, end_year=2024)
team_data = preprocessor.load_all_team_data(start_year=2020, end_year=2024)

print(f"\nData loaded:")
print(f"Player data shape: {player_data.shape}")
print(f"Team data shape: {team_data.shape}")

# Basic data quality check
print(f"\nPlayer data info:")
print(f"Date range: {player_data['season'].min()} - {player_data['season'].max()}")
print(f"Unique players: {player_data['player_id'].nunique()}")
print(f"Positions: {player_data['position'].value_counts()}")

print(f"\nTeam data info:")
print(f"Date range: {team_data['season'].min()} - {team_data['season'].max()}")
print(f"Unique teams: {team_data['team'].nunique()}")


Loading recent data for testing...


Loaded 2020: 17602 records
Loaded 2021: 18969 records
Loaded 2022: 18831 records
Loaded 2023: 18643 records
Loaded 2024: 18981 records
Total player records loaded: 93026
Loaded team data 2020: 538 records
Loaded team data 2021: 570 records
Loaded team data 2022: 568 records
Loaded team data 2023: 570 records
Loaded team data 2024: 570 records
Total team records loaded: 2816

Data loaded:
Player data shape: (93026, 114)
Team data shape: (2816, 102)

Player data info:
Date range: 2020 - 2024
Unique players: 3676
Positions: position
WR     12670
LB     10349
CB     10121
DE      8344
RB      7883
DT      7394
TE      6272
SAF     4595
QB      3384
K       2813
P       2777
DB      2664
OLB     2239
OT      2120
FS      1704
G       1545
S       1294
MLB     1080
ILB     1063
C        843
NT       592
FB       564
LS       357
DL       175
OL        75
Name: count, dtype: int64

Team data info:
Date range: 2020 - 2024
Unique teams: 32


In [8]:
# Let's examine the data structure more closely
print("Player data columns (first 20):")
print(player_data.columns[:20].tolist())

print("\nKey statistical columns:")
stat_cols = [col for col in player_data.columns if any(stat in col for stat in 
           ['yards', 'tds', 'completions', 'attempts', 'receptions', 'targets', 'carries'])]
print(stat_cols[:15])

print("\nSample QB data:")
qb_data = player_data[player_data['position'] == 'QB'].head()
print(qb_data[['player_name', 'season', 'week', 'team', 'passing_yards', 'passing_tds', 'completions', 'attempts']])


Player data columns (first 20):
['player_id', 'player_name', 'player_display_name', 'position', 'position_group', 'headshot_url', 'season', 'week', 'season_type', 'team', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost', 'sack_fumbles', 'sack_fumbles_lost']

Key statistical columns:
['completions', 'attempts', 'passing_yards', 'passing_tds', 'sack_yards_lost', 'passing_air_yards', 'passing_yards_after_catch', 'carries', 'rushing_yards', 'rushing_tds', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_air_yards']

Sample QB data:
        player_name  season  week team  passing_yards  passing_tds  \
0           T.Brady    2020     1   TB            239            2   
1           D.Brees    2020     1   NO            160            2   
5  B.Roethlisberger    2020     1  PIT            229            3   
6          P.Rivers    2020     1  IND            363            1   
8    

In [9]:
# Add methods to DataPreprocessor for feature engineering
def add_temporal_features(self, df):
    """Add temporal features like season progression, home/away, etc."""
    df = df.copy()
    
    # Create game identifier
    df['game_id'] = df['season'].astype(str) + '_' + df['week'].astype(str) + '_' + df['team']
    
    # Season progression (week as fraction of season)
    df['season_progression'] = df['week'] / 18.0  # Assuming 18-week season
    
    # Playoff indicator
    df['is_playoff'] = (df['season_type'] == 'POST').astype(int)
    
    # Home/away (we'll need to infer this from opponent data)
    # For now, we'll create a placeholder
    df['is_home'] = 0  # Placeholder - would need schedule data
    
    return df

def create_rolling_features(self, df, player_id_col='player_id', stat_cols=None, windows=[3, 5]):
    """Create rolling averages for key statistics."""
    if stat_cols is None:
        stat_cols = ['passing_yards', 'rushing_yards', 'receiving_yards', 'receptions', 'targets']
    
    df = df.sort_values(['player_id', 'season', 'week'])
    
    for window in windows:
        for stat in stat_cols:
            if stat in df.columns:
                # Rolling mean
                df[f'{stat}_avg_{window}'] = df.groupby('player_id')[stat].rolling(
                    window=window, min_periods=1
                ).mean().reset_index(0, drop=True)
                
                # Rolling std
                df[f'{stat}_std_{window}'] = df.groupby('player_id')[stat].rolling(
                    window=window, min_periods=1
                ).std().reset_index(0, drop=True)
    
    return df

def merge_team_context(self, player_df, team_df):
    """Merge team-level context features."""
    # Create team game identifier
    team_df = team_df.copy()
    team_df['team_game_id'] = team_df['season'].astype(str) + '_' + team_df['week'].astype(str) + '_' + team_df['team']
    
    # Select relevant team features (only numeric ones)
    team_features = ['passing_yards', 'rushing_yards', 'receptions', 'targets']
    team_subset = team_df[['team_game_id'] + team_features].copy()
    
    # Rename columns to indicate team context
    team_subset.columns = ['team_game_id'] + [f'team_{col}' for col in team_features]
    
    # Merge with player data
    player_df = player_df.merge(team_subset, left_on='game_id', right_on='team_game_id', how='left')
    
    # Drop the team_game_id column to avoid data type issues
    player_df = player_df.drop('team_game_id', axis=1)
    
    return player_df

# Add methods to the class
DataPreprocessor.add_temporal_features = add_temporal_features
DataPreprocessor.create_rolling_features = create_rolling_features
DataPreprocessor.merge_team_context = merge_team_context

print("Feature engineering methods added to DataPreprocessor!")


Feature engineering methods added to DataPreprocessor!


In [10]:
# Test the preprocessing pipeline on a small sample
print("Testing preprocessing pipeline...")

# Take a small sample for testing
sample_players = player_data[player_data['position'] == 'QB'].head(1000)
sample_teams = team_data.head(500)

print(f"Sample data shapes: Players {sample_players.shape}, Teams {sample_teams.shape}")

# Add temporal features
sample_players = preprocessor.add_temporal_features(sample_players)
print(f"After temporal features: {sample_players.shape}")

# Create rolling features for QBs
qb_stats = ['passing_yards', 'passing_tds', 'completions', 'attempts']
sample_players = preprocessor.create_rolling_features(sample_players, stat_cols=qb_stats, windows=[3])
print(f"After rolling features: {sample_players.shape}")

# Merge team context
sample_players = preprocessor.merge_team_context(sample_players, sample_teams)
print(f"After team context: {sample_players.shape}")

print("\nSample of processed data:")
print(sample_players[['player_name', 'season', 'week', 'passing_yards', 'passing_yards_avg_3', 'team_passing_yards']].head())


Testing preprocessing pipeline...
Sample data shapes: Players (1000, 114), Teams (500, 102)
After temporal features: (1000, 118)
After rolling features: (1000, 126)
After team context: (1000, 130)

Sample of processed data:
  player_name  season  week  passing_yards  passing_yards_avg_3  \
0     T.Brady    2020     1            239           239.000000   
1     T.Brady    2020     2            217           228.000000   
2     T.Brady    2020     3            297           251.000000   
3     T.Brady    2020     4            369           294.333333   
4     T.Brady    2020     5            253           306.333333   

   team_passing_yards  
0               239.0  
1               217.0  
2               297.0  
3               369.0  
4               253.0  


## 3. Model Training Pipeline

Now let's implement the core ML pipeline with per-position LightGBM quantile models for predicting player prop outcomes.


In [11]:
class ModelTrainer:
    """LightGBM quantile models for per-position player prop predictions."""
    
    def __init__(self):
        self.models = {}
        self.feature_columns = {}
        self.scalers = {}
        
    def prepare_features(self, df, position, target_stat):
        """Prepare features for a specific position and target statistic."""
        # Get position-specific stats
        if position == 'QB':
            feature_stats = ['passing_yards', 'passing_tds', 'completions', 'attempts', 'passing_interceptions']
        elif position == 'RB':
            feature_stats = ['rushing_yards', 'rushing_tds', 'carries', 'receptions', 'receiving_yards']
        elif position in ['WR', 'TE']:
            feature_stats = ['receiving_yards', 'receptions', 'receiving_tds', 'targets']
        else:
            feature_stats = []
        
        # Base features
        base_features = ['season_progression', 'is_playoff', 'is_home']
        
        # Rolling features
        rolling_features = []
        for stat in feature_stats:
            for window in [3, 5]:
                rolling_features.extend([f'{stat}_avg_{window}', f'{stat}_std_{window}'])
        
        # Team context features (only numeric ones)
        team_features = [col for col in df.columns if col.startswith('team_') and col != 'team_game_id']
        
        # Combine all features
        all_features = base_features + rolling_features + team_features
        
        # Filter to available features and ensure they're numeric
        available_features = []
        for f in all_features:
            if f in df.columns:
                # Check if the column is numeric
                if pd.api.types.is_numeric_dtype(df[f]):
                    available_features.append(f)
                else:
                    print(f"Warning: Skipping non-numeric feature: {f}")
        
        return available_features
    
    def train_position_model(self, df, position, target_stat, quantiles=[0.1, 0.5, 0.9]):
        """Train quantile models for a specific position and target statistic."""
        print(f"Training {position} model for {target_stat}...")
        
        # Filter data for position
        pos_data = df[df['position'] == position].copy()
        
        if len(pos_data) < 100:
            print(f"Warning: Not enough data for {position} ({len(pos_data)} records)")
            return None
        
        # Prepare features
        feature_cols = self.prepare_features(pos_data, position, target_stat)
        
        if len(feature_cols) == 0:
            print(f"Warning: No features available for {position}")
            return None
        
        # Prepare data
        X = pos_data[feature_cols].fillna(0)
        y = pos_data[target_stat].fillna(0)
        
        # Remove rows where target is missing
        valid_mask = ~y.isna()
        X = X[valid_mask]
        y = y[valid_mask]
        
        if len(X) < 50:
            print(f"Warning: Not enough valid data for {position} ({len(X)} records)")
            return None
        
        # Store feature columns
        self.feature_columns[f"{position}_{target_stat}"] = feature_cols
        
        # Train models for different quantiles
        position_models = {}
        
        for alpha in quantiles:
            print(f"  Training quantile {alpha}...")
            
            # LightGBM parameters
            params = {
                'objective': 'quantile',
                'alpha': alpha,
                'learning_rate': 0.05,
                'num_leaves': 31,
                'min_data_in_leaf': 20,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 5,
                'verbose': -1,
                'random_state': 42
            }
            
            # Create and train model
            train_data = lgb.Dataset(X, label=y)
            model = lgb.train(params, train_data, num_boost_round=100)
            
            position_models[alpha] = model
        
        self.models[f"{position}_{target_stat}"] = position_models
        print(f"  {position} {target_stat} model trained successfully!")
        
        return position_models

# Initialize model trainer
model_trainer = ModelTrainer()
print("ModelTrainer initialized!")


ModelTrainer initialized!


In [12]:
# Complete preprocessing pipeline function
def complete_preprocessing_pipeline(player_data, team_data, start_year=2020, end_year=2024):
    """Complete preprocessing pipeline for all data."""
    print("Starting complete preprocessing pipeline...")
    
    # Filter to recent years for testing
    player_data = player_data[(player_data['season'] >= start_year) & (player_data['season'] <= end_year)]
    team_data = team_data[(team_data['season'] >= start_year) & (team_data['season'] <= end_year)]
    
    print(f"Filtered data: Players {player_data.shape}, Teams {team_data.shape}")
    
    # Add temporal features
    print("Adding temporal features...")
    player_data = preprocessor.add_temporal_features(player_data)
    
    # Create rolling features for each position
    print("Creating rolling features...")
    positions = ['QB', 'RB', 'WR', 'TE']
    
    for position in positions:
        pos_data = player_data[player_data['position'] == position]
        if len(pos_data) > 0:
            print(f"  Processing {position} ({len(pos_data)} records)...")
            
            # Get position-specific stats
            if position == 'QB':
                stats = ['passing_yards', 'passing_tds', 'completions', 'attempts']
            elif position == 'RB':
                stats = ['rushing_yards', 'rushing_tds', 'carries', 'receptions', 'receiving_yards']
            elif position in ['WR', 'TE']:
                stats = ['receiving_yards', 'receptions', 'receiving_tds', 'targets']
            
            # Create rolling features
            pos_data = preprocessor.create_rolling_features(pos_data, stat_cols=stats, windows=[3, 5])
            
            # Update the main dataframe
            player_data.loc[player_data['position'] == position, pos_data.columns] = pos_data
    
    # Merge team context
    print("Merging team context...")
    player_data = preprocessor.merge_team_context(player_data, team_data)
    
    print(f"Preprocessing complete! Final shape: {player_data.shape}")
    return player_data

# Run the complete preprocessing pipeline
print("Running complete preprocessing pipeline...")
processed_data = complete_preprocessing_pipeline(player_data, team_data)

print(f"\nProcessed data summary:")
print(f"Shape: {processed_data.shape}")
print(f"Columns: {len(processed_data.columns)}")
print(f"Positions: {processed_data['position'].value_counts()}")
print(f"Years: {processed_data['season'].unique()}")

# Show sample of processed features
feature_cols = [col for col in processed_data.columns if any(x in col for x in ['_avg_', '_std_', 'team_', 'season_progression'])]
print(f"\nSample engineered features: {feature_cols[:10]}")


Running complete preprocessing pipeline...
Starting complete preprocessing pipeline...
Filtered data: Players (93026, 114), Teams (2816, 102)
Adding temporal features...
Creating rolling features...
  Processing QB (3384 records)...
  Processing RB (7883 records)...
  Processing WR (12670 records)...
  Processing TE (6272 records)...
Merging team context...
Preprocessing complete! Final shape: (93026, 166)

Processed data summary:
Shape: (93026, 166)
Columns: 166
Positions: position
WR     12670
LB     10349
CB     10121
DE      8344
RB      7883
DT      7394
TE      6272
SAF     4595
QB      3384
K       2813
P       2777
DB      2664
OLB     2239
OT      2120
FS      1704
G       1545
S       1294
MLB     1080
ILB     1063
C        843
NT       592
FB       564
LS       357
DL       175
OL        75
Name: count, dtype: int64
Years: [2020 2021 2022 2023 2024]

Sample engineered features: ['season_progression', 'passing_yards_avg_3', 'passing_yards_std_3', 'passing_tds_avg_3', 'passing

# Testing on QB players

In [13]:
# Test model training again with fixed preprocessing
print("Testing model training with fixed preprocessing...")

# Get QB data
qb_data = processed_data[processed_data['position'] == 'QB'].copy()
print(f"QB data shape: {qb_data.shape}")

if len(qb_data) > 100:
    # Train a QB passing yards model
    qb_model = model_trainer.train_position_model(qb_data, 'QB', 'passing_yards')
    
    if qb_model:
        print("✅ QB model training successful!")
        print(f"Available quantiles: {list(qb_model.keys())}")
        
        # Test prediction
        feature_cols = model_trainer.feature_columns['QB_passing_yards']
        print(f"Features used: {feature_cols}")
        
        sample_features = qb_data[feature_cols].fillna(0).iloc[:5]
        
        print(f"\nSample predictions for first 5 QBs:")
        for alpha, model in qb_model.items():
            predictions = model.predict(sample_features)
            print(f"Quantile {alpha}: {predictions}")
            
        # Show actual vs predicted for comparison
        actual_values = qb_data['passing_yards'].iloc[:5].values
        median_predictions = qb_model[0.5].predict(sample_features)
        print(f"\nActual vs Predicted (median):")
        for i in range(5):
            print(f"Player {i+1}: Actual={actual_values[i]:.1f}, Predicted={median_predictions[i]:.1f}")
    else:
        print("❌ QB model training failed!")
else:
    print("Not enough QB data for training")


Testing model training with fixed preprocessing...
QB data shape: (3384, 166)
Training QB model for passing_yards...
  Training quantile 0.1...
  Training quantile 0.5...
  Training quantile 0.9...
  QB passing_yards model trained successfully!
✅ QB model training successful!
Available quantiles: [0.1, 0.5, 0.9]
Features used: ['season_progression', 'is_playoff', 'is_home', 'passing_yards_avg_3', 'passing_yards_std_3', 'passing_yards_avg_5', 'passing_yards_std_5', 'passing_tds_avg_3', 'passing_tds_std_3', 'passing_tds_avg_5', 'passing_tds_std_5', 'completions_avg_3', 'completions_std_3', 'completions_avg_5', 'completions_std_5', 'attempts_avg_3', 'attempts_std_3', 'attempts_avg_5', 'attempts_std_5', 'team_passing_yards', 'team_rushing_yards', 'team_receptions', 'team_targets']

Sample predictions for first 5 QBs:
Quantile 0.1: [223.4110042  155.07472993 213.71078534 279.46583928 279.46583928]
Quantile 0.5: [239.33534605 191.32535316 229.20478219 359.31934774 364.5018578 ]
Quantile 0.9:

# Testing on RB players

In [14]:
# Test model training for RB players
print("Testing model training for RB rushing yards...")

# Get RB data
rb_data = processed_data[processed_data['position'] == 'RB'].copy()
print(f"RB data shape: {rb_data.shape}")

if len(rb_data) > 100:
    # Train a RB rushing yards model
    rb_model = model_trainer.train_position_model(rb_data, 'RB', 'rushing_yards')
    
    if rb_model:
        print("✅ RB model training successful!")
        print(f"Available quantiles: {list(rb_model.keys())}")
        
        # Test prediction
        feature_cols = model_trainer.feature_columns['RB_rushing_yards']
        print(f"Features used: {feature_cols}")
        
        sample_features = rb_data[feature_cols].fillna(0).iloc[:5]
        
        print(f"\nSample predictions for first 5 RBs:")
        for alpha, model in rb_model.items():
            predictions = model.predict(sample_features)
            print(f"Quantile {alpha}: {predictions}")
            
        # Show actual vs predicted for comparison
        actual_values = rb_data['rushing_yards'].iloc[:5].values
        median_predictions = rb_model[0.5].predict(sample_features)
        print(f"\nActual vs Predicted (median):")
        for i in range(5):
            print(f"Player {i+1}: Actual={actual_values[i]:.1f}, Predicted={median_predictions[i]:.1f}")
            
        # Show some RB names for context
        print(f"\nRB names for context:")
        rb_names = rb_data[['player_name', 'season', 'week', 'team', 'rushing_yards']].iloc[:5]
        for i, row in rb_names.iterrows():
            print(f"{row['player_name']} ({row['season']} W{row['week']} {row['team']}): {row['rushing_yards']} yards")
    else:
        print("❌ RB model training failed!")
else:
    print("Not enough RB data for training")


Testing model training for RB rushing yards...
RB data shape: (7883, 166)
Training RB model for rushing_yards...
  Training quantile 0.1...
  Training quantile 0.5...
  Training quantile 0.9...
  RB rushing_yards model trained successfully!
✅ RB model training successful!
Available quantiles: [0.1, 0.5, 0.9]
Features used: ['season_progression', 'is_playoff', 'is_home', 'rushing_yards_avg_3', 'rushing_yards_std_3', 'rushing_yards_avg_5', 'rushing_yards_std_5', 'rushing_tds_avg_3', 'rushing_tds_std_3', 'rushing_tds_avg_5', 'rushing_tds_std_5', 'carries_avg_3', 'carries_std_3', 'carries_avg_5', 'carries_std_5', 'receptions_avg_3', 'receptions_std_3', 'receptions_avg_5', 'receptions_std_5', 'receiving_yards_avg_3', 'receiving_yards_std_3', 'receiving_yards_avg_5', 'receiving_yards_std_5', 'team_passing_yards', 'team_rushing_yards', 'team_receptions', 'team_targets']

Sample predictions for first 5 RBs:
Quantile 0.1: [15.48949427 51.44905816  0.         22.27390728  0.80480833]
Quantile 0.

# Testing on WR players

In [15]:
# Test model training for WR players
print("Testing model training for WR receiving yards...")

# Get WR data
wr_data = processed_data[processed_data['position'] == 'WR'].copy()
print(f"WR data shape: {wr_data.shape}")

if len(wr_data) > 100:
    # Train a WR receiving yards model
    wr_model = model_trainer.train_position_model(wr_data, 'WR', 'receiving_yards')
    
    if wr_model:
        print("✅ WR model training successful!")
        print(f"Available quantiles: {list(wr_model.keys())}")
        
        # Test prediction
        feature_cols = model_trainer.feature_columns['WR_receiving_yards']
        print(f"Features used: {feature_cols}")
        
        sample_features = wr_data[feature_cols].fillna(0).iloc[:5]
        
        print(f"\nSample predictions for first 5 WRs:")
        for alpha, model in wr_model.items():
            predictions = model.predict(sample_features)
            print(f"Quantile {alpha}: {predictions}")
            
        # Show actual vs predicted for comparison
        actual_values = wr_data['receiving_yards'].iloc[:5].values
        median_predictions = wr_model[0.5].predict(sample_features)
        print(f"\nActual vs Predicted (median):")
        for i in range(5):
            print(f"Player {i+1}: Actual={actual_values[i]:.1f}, Predicted={median_predictions[i]:.1f}")
            
        # Show some WR names for context
        print(f"\nWR names for context:")
        wr_names = wr_data[['player_name', 'season', 'week', 'team', 'receiving_yards']].iloc[:5]
        for i, row in wr_names.iterrows():
            print(f"{row['player_name']} ({row['season']} W{row['week']} {row['team']}): {row['receiving_yards']} yards")
    else:
        print("❌ WR model training failed!")
else:
    print("Not enough WR data for training")


Testing model training for WR receiving yards...
WR data shape: (12670, 166)
Training WR model for receiving_yards...
  Training quantile 0.1...
  Training quantile 0.5...
  Training quantile 0.9...
  WR receiving_yards model trained successfully!
✅ WR model training successful!
Available quantiles: [0.1, 0.5, 0.9]
Features used: ['season_progression', 'is_playoff', 'is_home', 'receiving_yards_avg_3', 'receiving_yards_std_3', 'receiving_yards_avg_5', 'receiving_yards_std_5', 'receptions_avg_3', 'receptions_std_3', 'receptions_avg_5', 'receptions_std_5', 'receiving_tds_avg_3', 'receiving_tds_std_3', 'receiving_tds_avg_5', 'receiving_tds_std_5', 'targets_avg_3', 'targets_std_3', 'targets_avg_5', 'targets_std_5', 'team_passing_yards', 'team_rushing_yards', 'team_receptions', 'team_targets']

Sample predictions for first 5 WRs:
Quantile 0.1: [26.2764613   0.         47.11776055 27.88461741 40.57257287]
Quantile 0.5: [33.85800957  0.13617217 81.3516545  45.70438828 57.47918917]
Quantile 0.9

# Testing on TE players

In [16]:
# Test model training for TE players
print("Testing model training for TE receiving yards...")

# Get TE data
te_data = processed_data[processed_data['position'] == 'TE'].copy()
print(f"TE data shape: {te_data.shape}")

if len(te_data) > 100:
    # Train a TE receiving yards model
    te_model = model_trainer.train_position_model(te_data, 'TE', 'receiving_yards')
    
    if te_model:
        print("✅ TE model training successful!")
        print(f"Available quantiles: {list(te_model.keys())}")
        
        # Test prediction
        feature_cols = model_trainer.feature_columns['TE_receiving_yards']
        print(f"Features used: {feature_cols}")
        
        sample_features = te_data[feature_cols].fillna(0).iloc[:5]
        
        print(f"\nSample predictions for first 5 TEs:")
        for alpha, model in te_model.items():
            predictions = model.predict(sample_features)
            print(f"Quantile {alpha}: {predictions}")
            
        # Show actual vs predicted for comparison
        actual_values = te_data['receiving_yards'].iloc[:5].values
        median_predictions = te_model[0.5].predict(sample_features)
        print(f"\nActual vs Predicted (median):")
        for i in range(5):
            print(f"Player {i+1}: Actual={actual_values[i]:.1f}, Predicted={median_predictions[i]:.1f}")
            
        # Show some TE names for context
        print(f"\nTE names for context:")
        te_names = te_data[['player_name', 'season', 'week', 'team', 'receiving_yards']].iloc[:5]
        for i, row in te_names.iterrows():
            print(f"{row['player_name']} ({row['season']} W{row['week']} {row['team']}): {row['receiving_yards']} yards")
    else:
        print("❌ TE model training failed!")
else:
    print("Not enough TE data for training")


Testing model training for TE receiving yards...
TE data shape: (6272, 166)
Training TE model for receiving_yards...
  Training quantile 0.1...
  Training quantile 0.5...
  Training quantile 0.9...
  TE receiving_yards model trained successfully!
✅ TE model training successful!
Available quantiles: [0.1, 0.5, 0.9]
Features used: ['season_progression', 'is_playoff', 'is_home', 'receiving_yards_avg_3', 'receiving_yards_std_3', 'receiving_yards_avg_5', 'receiving_yards_std_5', 'receptions_avg_3', 'receptions_std_3', 'receptions_avg_5', 'receptions_std_5', 'receiving_tds_avg_3', 'receiving_tds_std_3', 'receiving_tds_avg_5', 'receiving_tds_std_5', 'targets_avg_3', 'targets_std_3', 'targets_avg_5', 'targets_std_5', 'team_passing_yards', 'team_rushing_yards', 'team_receptions', 'team_targets']

Sample predictions for first 5 TEs:
Quantile 0.1: [ 0.54666473 17.41584431 30.93322363  7.74989816 16.57240838]
Quantile 0.5: [ 1.88864723 25.48091169 62.709778   11.07553146 24.97856811]
Quantile 0.9:

In [17]:
# ============================================================================
# STEP 1: Check data availability for all stats before training
# ============================================================================

position_stats_check = {
    'QB': ['passing_yards', 'passing_tds', 'completions', 'passing_interceptions', 'rushing_yards'],
    'RB': ['rushing_yards', 'rushing_tds', 'receptions', 'receiving_yards', 'receiving_tds'],
    'WR': ['receiving_yards', 'receptions', 'receiving_tds', 'targets'],
    'TE': ['receiving_yards', 'receptions', 'receiving_tds']
}

print("="*80)
print("CHECKING DATA AVAILABILITY FOR ALL BETTING STATS")
print("="*80 + "\n")

available_configs = []

for position, stats in position_stats_check.items():
    pos_data = processed_data[processed_data['position'] == position]
    print(f"📊 {position} Position ({len(pos_data):,} total records):")
    print("-" * 80)
    
    for stat in stats:
        if stat in pos_data.columns:
            # Check how many non-null and non-zero values
            non_null = pos_data[stat].notna().sum()
            non_zero = (pos_data[stat] > 0).sum()
            pct_non_zero = (non_zero / len(pos_data) * 100) if len(pos_data) > 0 else 0
            
            # Calculate some basic stats
            mean_val = pos_data[pos_data[stat] > 0][stat].mean()
            
            status = "✅ READY" if non_zero > 100 else "⚠️ LIMITED DATA"
            print(f"  {status:15} {stat:25} | Non-zero: {non_zero:5,} ({pct_non_zero:5.1f}%) | Avg: {mean_val:.1f}")
            
            if non_zero > 100:
                available_configs.append((position, stat))
        else:
            print(f"  ❌ NOT FOUND   {stat:25} | Column does not exist in data")
    
    print()

print("="*80)
print(f"✅ Total trainable position-stat combinations: {len(available_configs)}")
print("="*80)


CHECKING DATA AVAILABILITY FOR ALL BETTING STATS

📊 QB Position (3,384 total records):
--------------------------------------------------------------------------------
  ✅ READY         passing_yards             | Non-zero: 3,161 ( 93.4%) | Avg: 213.5
  ✅ READY         passing_tds               | Non-zero: 2,273 ( 67.2%) | Avg: 1.8
  ✅ READY         completions               | Non-zero: 3,168 ( 93.6%) | Avg: 19.4
  ✅ READY         passing_interceptions     | Non-zero: 1,501 ( 44.4%) | Avg: 1.4
  ✅ READY         rushing_yards             | Non-zero: 2,435 ( 72.0%) | Avg: 21.6

📊 RB Position (7,883 total records):
--------------------------------------------------------------------------------
  ✅ READY         rushing_yards             | Non-zero: 6,459 ( 81.9%) | Avg: 40.7
  ✅ READY         rushing_tds               | Non-zero: 1,575 ( 20.0%) | Avg: 1.2
  ✅ READY         receptions                | Non-zero: 5,082 ( 64.5%) | Avg: 2.5
  ✅ READY         receiving_yards           | Non-ze

In [18]:
# ============================================================================
# STEP 2: Train ALL models for complete betting coverage
# ============================================================================

print("\n" + "="*80)
print("TRAINING COMPLETE MODEL SUITE")
print("="*80 + "\n")

# Define all position-stat combinations to train
all_model_configs = [
    # QB models
    ('QB', 'passing_yards'),
    ('QB', 'passing_tds'),
    ('QB', 'completions'),
    ('QB', 'passing_interceptions'),
    ('QB', 'rushing_yards'),
    
    # RB models  
    ('RB', 'rushing_yards'),
    ('RB', 'rushing_tds'),
    ('RB', 'receptions'),
    ('RB', 'receiving_yards'),
    ('RB', 'receiving_tds'),
    
    # WR models
    ('WR', 'receiving_yards'),
    ('WR', 'receptions'),
    ('WR', 'receiving_tds'),
    ('WR', 'targets'),
    
    # TE models
    ('TE', 'receiving_yards'),
    ('TE', 'receptions'),
    ('TE', 'receiving_tds'),
]

print(f"📋 Configured to train {len(all_model_configs)} models")
print(f"🔄 Training with quantiles: [0.1, 0.5, 0.9]")
print(f"💾 Each model will be saved for production use\n")

# Track training results
training_results = []
start_time = pd.Timestamp.now()

# Train each model
for idx, (position, stat) in enumerate(all_model_configs, 1):
    print(f"\n{'='*80}")
    print(f"🏈 Model {idx}/{len(all_model_configs)}: {position} - {stat}")
    print(f"{'='*80}")
    
    # Get position data
    pos_data = processed_data[processed_data['position'] == position].copy()
    
    # Check if we have enough data
    if len(pos_data) < 100:
        print(f"⚠️  SKIPPED: Insufficient data ({len(pos_data)} records, need at least 100)")
        training_results.append({
            'position': position,
            'stat': stat,
            'status': 'SKIPPED',
            'reason': 'Insufficient data',
            'records': len(pos_data),
            'models_trained': 0
        })
        continue
    
    # Check if stat column exists and has data
    if stat not in pos_data.columns:
        print(f"⚠️  SKIPPED: Column '{stat}' not found in data")
        training_results.append({
            'position': position,
            'stat': stat,
            'status': 'SKIPPED',
            'reason': 'Column not found',
            'records': len(pos_data),
            'models_trained': 0
        })
        continue
    
    # Check for non-null values
    valid_records = pos_data[stat].notna().sum()
    if valid_records < 50:
        print(f"⚠️  SKIPPED: Insufficient valid data ({valid_records} non-null records)")
        training_results.append({
            'position': position,
            'stat': stat,
            'status': 'SKIPPED',
            'reason': 'Too few valid records',
            'records': valid_records,
            'models_trained': 0
        })
        continue
    
    # Train the model
    try:
        print(f"📊 Training on {len(pos_data):,} records ({valid_records:,} valid)")
        
        model = model_trainer.train_position_model(pos_data, position, stat)
        
        if model and len(model) == 3:
            # Get some stats about the data
            stat_mean = pos_data[stat].mean()
            stat_median = pos_data[stat].median()
            stat_std = pos_data[stat].std()
            
            print(f"✅ SUCCESS!")
            print(f"   📈 Data stats: Mean={stat_mean:.1f}, Median={stat_median:.1f}, StdDev={stat_std:.1f}")
            print(f"   🎯 Trained 3 quantile models (10th, 50th, 90th percentiles)")
            
            training_results.append({
                'position': position,
                'stat': stat,
                'status': 'SUCCESS',
                'records': len(pos_data),
                'valid_records': valid_records,
                'mean': stat_mean,
                'median': stat_median,
                'std': stat_std,
                'models_trained': 3,
                'reason': None
            })
        else:
            print(f"❌ FAILED: Training function did not return expected models")
            training_results.append({
                'position': position,
                'stat': stat,
                'status': 'FAILED',
                'reason': 'No models returned',
                'records': len(pos_data),
                'models_trained': 0
            })
            
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        training_results.append({
            'position': position,
            'stat': stat,
            'status': 'ERROR',
            'reason': str(e)[:100],
            'records': len(pos_data),
            'models_trained': 0
        })

# Calculate training time
end_time = pd.Timestamp.now()
duration = (end_time - start_time).total_seconds()

print("\n" + "="*80)
print("📊 TRAINING COMPLETE!")
print("="*80)



TRAINING COMPLETE MODEL SUITE

📋 Configured to train 17 models
🔄 Training with quantiles: [0.1, 0.5, 0.9]
💾 Each model will be saved for production use


🏈 Model 1/17: QB - passing_yards
📊 Training on 3,384 records (3,384 valid)
Training QB model for passing_yards...
  Training quantile 0.1...
  Training quantile 0.5...
  Training quantile 0.9...
  QB passing_yards model trained successfully!
✅ SUCCESS!
   📈 Data stats: Mean=199.4, Median=214.0, StdDev=105.6
   🎯 Trained 3 quantile models (10th, 50th, 90th percentiles)

🏈 Model 2/17: QB - passing_tds
📊 Training on 3,384 records (3,384 valid)
Training QB model for passing_tds...
  Training quantile 0.1...
  Training quantile 0.5...
  Training quantile 0.9...
  QB passing_tds model trained successfully!
✅ SUCCESS!
   📈 Data stats: Mean=1.2, Median=1.0, StdDev=1.2
   🎯 Trained 3 quantile models (10th, 50th, 90th percentiles)

🏈 Model 3/17: QB - completions
📊 Training on 3,384 records (3,384 valid)
Training QB model for completions...
  T

In [19]:
# ============================================================================
# STEP 3: Display comprehensive training summary
# ============================================================================

# Create DataFrame from results
results_df = pd.DataFrame(training_results)

# Display summary statistics
print(f"\n⏱️  Total Training Time: {duration/60:.1f} minutes ({duration:.0f} seconds)")
print(f"📦 Total Models Configured: {len(all_model_configs)}")

# Count by status
status_counts = results_df['status'].value_counts()
print(f"\n📊 Training Results by Status:")
print("-" * 80)
for status, count in status_counts.items():
    percentage = (count / len(results_df) * 100)
    if status == 'SUCCESS':
        print(f"   ✅ {status:12} : {count:2} models ({percentage:5.1f}%)")
    elif status == 'SKIPPED':
        print(f"   ⚠️  {status:12} : {count:2} models ({percentage:5.1f}%)")
    else:
        print(f"   ❌ {status:12} : {count:2} models ({percentage:5.1f}%)")

# Calculate total quantile models
total_quantile_models = results_df['models_trained'].sum()
print(f"\n🎯 Total Quantile Models Trained: {total_quantile_models} (each position-stat has 3 quantiles)")

# Display successful models
success_df = results_df[results_df['status'] == 'SUCCESS'].copy()
if len(success_df) > 0:
    print(f"\n{'='*80}")
    print(f"✅ SUCCESSFULLY TRAINED MODELS ({len(success_df)} position-stat combinations)")
    print(f"{'='*80}")
    
    # Group by position
    for position in ['QB', 'RB', 'WR', 'TE']:
        pos_models = success_df[success_df['position'] == position]
        if len(pos_models) > 0:
            print(f"\n{position} Models ({len(pos_models)}):")
            for _, row in pos_models.iterrows():
                print(f"   ✅ {row['stat']:25} | Records: {row['records']:5,} | "
                      f"Mean: {row['mean']:6.1f} | Median: {row['median']:6.1f}")

# Display failed/skipped models
failed_df = results_df[results_df['status'].isin(['FAILED', 'SKIPPED', 'ERROR'])].copy()
if len(failed_df) > 0:
    print(f"\n{'='*80}")
    print(f"⚠️  FAILED/SKIPPED MODELS ({len(failed_df)})")
    print(f"{'='*80}")
    for _, row in failed_df.iterrows():
        print(f"   {row['status']:8} | {row['position']:3} - {row['stat']:25} | Reason: {row['reason']}")

# Display model inventory
print(f"\n{'='*80}")
print(f"📦 MODEL INVENTORY - Ready for Production")
print(f"{'='*80}")
print(f"Total trained model keys: {len(model_trainer.models)}")
print(f"Models stored in memory:")
for model_key in sorted(model_trainer.models.keys()):
    quantiles = list(model_trainer.models[model_key].keys())
    print(f"   ✅ {model_key:30} | Quantiles: {quantiles}")

print(f"\n{'='*80}")
print(f"🎉 MODEL TRAINING COMPLETE - Ready for Phase 1 Export!")
print(f"{'='*80}")



⏱️  Total Training Time: 0.2 minutes (13 seconds)
📦 Total Models Configured: 17

📊 Training Results by Status:
--------------------------------------------------------------------------------
   ✅ SUCCESS      : 17 models (100.0%)

🎯 Total Quantile Models Trained: 51 (each position-stat has 3 quantiles)

✅ SUCCESSFULLY TRAINED MODELS (17 position-stat combinations)

QB Models (5):
   ✅ passing_yards             | Records: 3,384 | Mean:  199.4 | Median:  214.0
   ✅ passing_tds               | Records: 3,384 | Mean:    1.2 | Median:    1.0
   ✅ completions               | Records: 3,384 | Mean:   18.2 | Median:   20.0
   ✅ passing_interceptions     | Records: 3,384 | Mean:    0.6 | Median:    0.0
   ✅ rushing_yards             | Records: 3,384 | Mean:   15.3 | Median:    7.0

RB Models (5):
   ✅ rushing_yards             | Records: 7,883 | Mean:   33.3 | Median:   22.0
   ✅ rushing_tds               | Records: 7,883 | Mean:    0.2 | Median:    0.0
   ✅ receptions                | Record

In [22]:
# ============================================================================
# PHASE 1: EXPORT MODELS - Save all trained models to disk
# ============================================================================

print("\n" + "="*80)
print("PHASE 1: EXPORTING MODELS TO DISK")
print("="*80 + "\n")

# Create directory for saved models
model_dir = Path('../saved_models')
model_dir.mkdir(exist_ok=True)

print(f"📁 Created export directory: {model_dir.absolute()}\n")

# Step 1: Save all quantile models
print("STEP 1: Saving individual quantile models...")
print("-" * 80)

saved_files = []

for model_key, quantile_models in model_trainer.models.items():
    # model_key is like 'QB_passing_yards'
    position, stat = model_key.split('_', 1)
    
    # Save each quantile model (0.1, 0.5, 0.9)
    for quantile, model in quantile_models.items():
        # Create filename like: QB_passing_yards_q10.joblib
        filename = f"{model_key}_q{int(quantile*100)}.joblib"
        filepath = model_dir / filename
        
        # Save the model using joblib
        joblib.dump(model, filepath)
        
        # Get file size
        file_size_kb = filepath.stat().st_size / 1024
        
        saved_files.append({
            'position': position,
            'stat': stat.replace('_', ' '),
            'quantile': quantile,
            'filename': filename,
            'size_kb': file_size_kb
        })
        
        print(f"✅ Saved: {filename:40} ({file_size_kb:6.1f} KB)")

print(f"\n✅ Saved {len(saved_files)} quantile model files")

# Step 2: Save feature columns
print(f"\nSTEP 2: Saving feature column metadata...")
print("-" * 80)

feature_cols_path = model_dir / 'feature_columns.joblib'
joblib.dump(model_trainer.feature_columns, feature_cols_path)
file_size_kb = feature_cols_path.stat().st_size / 1024
print(f"✅ Saved: feature_columns.joblib ({file_size_kb:.1f} KB)")
print(f"   Contains feature lists for {len(model_trainer.feature_columns)} models")

# Step 3: Save model metadata
print(f"\nSTEP 3: Saving model metadata...")
print("-" * 80)

metadata = {
    'training_date': datetime.now().isoformat(),
    'data_years': '2020-2024',
    'total_records_processed': len(processed_data),
    'models_trained': list(model_trainer.models.keys()),
    'quantiles': [0.1, 0.5, 0.9],
    'positions': ['QB', 'RB', 'WR', 'TE'],
    'model_details': {},
    'training_summary': results_df.to_dict('records')
}

# Add details for each successful model
for _, row in success_df.iterrows():
    key = f"{row['position']}_{row['stat']}"
    metadata['model_details'][key] = {
        'position': row['position'],
        'stat': row['stat'],
        'training_records': int(row['records']),
        'valid_records': int(row['valid_records']),
        'mean': float(row['mean']),
        'median': float(row['median']),
        'std': float(row['std']),
        'quantiles_trained': 3
    }

metadata_path = model_dir / 'model_metadata.joblib'
joblib.dump(metadata, metadata_path)
file_size_kb = metadata_path.stat().st_size / 1024
print(f"✅ Saved: model_metadata.joblib ({file_size_kb:.1f} KB)")
print(f"   Contains metadata for {len(metadata['model_details'])} models")



PHASE 1: EXPORTING MODELS TO DISK

📁 Created export directory: c:\Users\tonyg\OneDrive\Desktop\University Files\Fall 2025\SWE\Main Project\hedge-your-bets\hedge-your-bets\machine_learning\training_code\..\saved_models

STEP 1: Saving individual quantile models...
--------------------------------------------------------------------------------
✅ Saved: QB_passing_yards_q10.joblib              ( 274.4 KB)
✅ Saved: QB_passing_yards_q50.joblib              ( 274.9 KB)
✅ Saved: QB_passing_yards_q90.joblib              ( 256.4 KB)
✅ Saved: RB_rushing_yards_q10.joblib              ( 278.3 KB)
✅ Saved: RB_rushing_yards_q50.joblib              ( 278.7 KB)
✅ Saved: RB_rushing_yards_q90.joblib              ( 277.0 KB)
✅ Saved: WR_receiving_yards_q10.joblib            ( 266.4 KB)
✅ Saved: WR_receiving_yards_q50.joblib            ( 279.8 KB)
✅ Saved: WR_receiving_yards_q90.joblib            ( 277.2 KB)
✅ Saved: TE_receiving_yards_q10.joblib            ( 271.6 KB)
✅ Saved: TE_receiving_yards_q50.jo

In [23]:
# Step 4: Export preprocessing code as Python module
print(f"\nSTEP 4: Exporting preprocessing code...")
print("-" * 80)

preprocessing_code = '''"""
Preprocessing utilities for Hedge Your Bets ML inference
Auto-generated from model_training.ipynb
Training Date: {training_date}
Data: 2020-2024 NFL player and team stats
"""

import pandas as pd
import numpy as np
from pathlib import Path


class InferencePreprocessor:
    """Preprocessing pipeline for making predictions on new betting scenarios."""
    
    def __init__(self, datasets_dir='../datasets'):
        """Initialize with path to datasets directory."""
        self.datasets_dir = Path(datasets_dir)
        self.player_weekly_dir = self.datasets_dir / "player_weekly_stats"
        self.team_weekly_dir = self.datasets_dir / "team_season_stats"
        
    def add_temporal_features(self, df):
        """Add temporal features like season progression."""
        df = df.copy()
        df['season_progression'] = df['week'] / 18.0
        df['is_playoff'] = (df['season_type'] == 'POST').astype(int)
        df['is_home'] = 0  # Placeholder - would need schedule data for actual home/away
        return df
    
    def create_rolling_features(self, df, stat_cols, windows=[3, 5]):
        """Create rolling averages for key statistics."""
        df = df.sort_values(['player_id', 'season', 'week'])
        
        for window in windows:
            for stat in stat_cols:
                if stat in df.columns:
                    # Rolling mean
                    df[f'{{stat}}_avg_{{window}}'] = df.groupby('player_id')[stat].rolling(
                        window=window, min_periods=1
                    ).mean().reset_index(0, drop=True)
                    
                    # Rolling std
                    df[f'{{stat}}_std_{{window}}'] = df.groupby('player_id')[stat].rolling(
                        window=window, min_periods=1
                    ).std().reset_index(0, drop=True)
        
        return df
    
    def merge_team_context(self, player_df, team_df):
        """Merge team-level context features."""
        team_df = team_df.copy()
        team_df['team_game_id'] = team_df['season'].astype(str) + '_' + team_df['week'].astype(str) + '_' + team_df['team']
        
        # Select relevant team features (only numeric ones)
        team_features = ['passing_yards', 'rushing_yards', 'receptions', 'targets']
        team_subset = team_df[['team_game_id'] + team_features].copy()
        
        # Rename columns to indicate team context
        team_subset.columns = ['team_game_id'] + [f'team_{{col}}' for col in team_features]
        
        # Create game identifier in player data
        player_df['game_id'] = player_df['season'].astype(str) + '_' + player_df['week'].astype(str) + '_' + player_df['team']
        
        # Merge with player data
        player_df = player_df.merge(team_subset, left_on='game_id', right_on='team_game_id', how='left')
        
        # Drop the team_game_id column
        if 'team_game_id' in player_df.columns:
            player_df = player_df.drop('team_game_id', axis=1)
        
        return player_df
    
    def get_player_recent_games(self, player_id, season, week, num_games=5):
        """
        Get recent game data for a player to calculate rolling features.
        This is the main function you'll use for inference.
        """
        # Load recent data (this is simplified - in production you'd have this data ready)
        # For now, assumes data is loaded from CSVs
        pass  # Implement based on your data loading strategy


# Position-specific stat mappings
POSITION_STATS = {{
    'QB': ['passing_yards', 'passing_tds', 'completions', 'attempts', 'passing_interceptions', 'rushing_yards'],
    'RB': ['rushing_yards', 'rushing_tds', 'carries', 'receptions', 'receiving_yards', 'receiving_tds'],
    'WR': ['receiving_yards', 'receptions', 'receiving_tds', 'targets'],
    'TE': ['receiving_yards', 'receptions', 'receiving_tds']
}}

# Action name mappings (frontend -> model stat name)
ACTION_TO_STAT = {{
    'Passing Yards': 'passing_yards',
    'Passing TDs': 'passing_tds',
    'Completions': 'completions',
    'Interceptions': 'passing_interceptions',
    'Rushing Yards': 'rushing_yards',
    'Rushing TDs': 'rushing_tds',
    'Receiving Yards': 'receiving_yards',
    'Receiving TDs': 'receiving_tds',
    'Receptions': 'receptions',
    'Targets': 'targets',
    'Touchdowns': 'touchdowns'  # Note: May need position-specific handling
}}

# Stat to action name (reverse mapping)
STAT_TO_ACTION = {{v: k for k, v in ACTION_TO_STAT.items()}}
'''.format(training_date=datetime.now().isoformat())

# Write to file
preprocessing_file = model_dir / 'inference_preprocessing.py'
with open(preprocessing_file, 'w') as f:
    f.write(preprocessing_code)

file_size_kb = preprocessing_file.stat().st_size / 1024
print(f"✅ Saved: inference_preprocessing.py ({file_size_kb:.1f} KB)")
print(f"   Contains InferencePreprocessor class and utility mappings")



STEP 4: Exporting preprocessing code...
--------------------------------------------------------------------------------
✅ Saved: inference_preprocessing.py (4.5 KB)
   Contains InferencePreprocessor class and utility mappings


In [24]:
# Step 5: Test loading models to verify they work
print(f"\nSTEP 5: Testing model loading...")
print("-" * 80)

# Test loading a few models
test_models = [
    ('QB_passing_yards_q50.joblib', 'QB Passing Yards Median'),
    ('RB_rushing_yards_q50.joblib', 'RB Rushing Yards Median'),
    ('WR_receiving_yards_q50.joblib', 'WR Receiving Yards Median')
]

all_tests_passed = True

for filename, description in test_models:
    filepath = model_dir / filename
    
    if filepath.exists():
        try:
            # Load the model
            loaded_model = joblib.load(filepath)
            
            # Make a simple test prediction
            # Get the position and stat from filename
            parts = filename.replace('.joblib', '').split('_')
            position = parts[0]
            stat = '_'.join(parts[1:-1])  # Everything except position and quantile
            
            # Get sample data
            pos_data = processed_data[processed_data['position'] == position]
            if len(pos_data) > 0:
                feature_cols = model_trainer.feature_columns[f'{position}_{stat}']
                test_features = pos_data[feature_cols].fillna(0).iloc[0:1]
                test_prediction = loaded_model.predict(test_features)
                
                print(f"✅ {description:35} | Loaded & tested | Prediction: {test_prediction[0]:.1f}")
            else:
                print(f"⚠️  {description:35} | Loaded but no test data")
                
        except Exception as e:
            print(f"❌ {description:35} | FAILED: {str(e)[:50]}")
            all_tests_passed = False
    else:
        print(f"❌ {description:35} | File not found")
        all_tests_passed = False

if all_tests_passed:
    print(f"\n✅ All model load tests passed!")
else:
    print(f"\n⚠️  Some tests failed - check errors above")

# Test loading metadata
print(f"\nTesting metadata loading...")
try:
    loaded_metadata = joblib.load(metadata_path)
    print(f"✅ Metadata loaded successfully")
    print(f"   Training date: {loaded_metadata['training_date']}")
    print(f"   Models: {len(loaded_metadata['models_trained'])}")
except Exception as e:
    print(f"❌ Metadata load failed: {str(e)}")



STEP 5: Testing model loading...
--------------------------------------------------------------------------------
✅ QB Passing Yards Median             | Loaded & tested | Prediction: 239.3
✅ RB Rushing Yards Median             | Loaded & tested | Prediction: 23.9
✅ WR Receiving Yards Median           | Loaded & tested | Prediction: 33.9

✅ All model load tests passed!

Testing metadata loading...
✅ Metadata loaded successfully
   Training date: 2025-10-19T22:52:11.336151
   Models: 17


In [25]:
# ============================================================================
# FINAL SUMMARY: Complete Export Report
# ============================================================================

print("\n" + "="*80)
print("🎉 PHASE 1 COMPLETE: MODELS EXPORTED SUCCESSFULLY!")
print("="*80 + "\n")

# Directory summary
print("📁 Export Directory:")
print(f"   {model_dir.absolute()}\n")

# List all files
all_files = list(model_dir.glob('*'))
print(f"📦 Exported Files ({len(all_files)} total):\n")

# Group files by type
model_files = [f for f in all_files if f.suffix == '.joblib' and 'q' in f.stem]
metadata_files = [f for f in all_files if f.suffix == '.joblib' and 'q' not in f.stem]
python_files = [f for f in all_files if f.suffix == '.py']

print(f"   Quantile Models: {len(model_files)}")
for f in sorted(model_files)[:5]:  # Show first 5
    size_kb = f.stat().st_size / 1024
    print(f"      • {f.name:40} ({size_kb:6.1f} KB)")
if len(model_files) > 5:
    print(f"      ... and {len(model_files) - 5} more model files")

print(f"\n   Metadata Files: {len(metadata_files)}")
for f in sorted(metadata_files):
    size_kb = f.stat().st_size / 1024
    print(f"      • {f.name:40} ({size_kb:6.1f} KB)")

print(f"\n   Python Modules: {len(python_files)}")
for f in sorted(python_files):
    size_kb = f.stat().st_size / 1024
    print(f"      • {f.name:40} ({size_kb:6.1f} KB)")

# Calculate total size
total_size_kb = sum(f.stat().st_size for f in all_files) / 1024
total_size_mb = total_size_kb / 1024
print(f"\n   💾 Total Size: {total_size_mb:.2f} MB ({total_size_kb:.1f} KB)")

# Model coverage summary
print(f"\n{'='*80}")
print("🏈 MODEL COVERAGE - What Can Be Predicted")
print("="*80 + "\n")

coverage_summary = {}
for model_key in sorted(model_trainer.models.keys()):
    position, stat = model_key.split('_', 1)
    if position not in coverage_summary:
        coverage_summary[position] = []
    coverage_summary[position].append(stat.replace('_', ' ').title())

for position in ['QB', 'RB', 'WR', 'TE']:
    if position in coverage_summary:
        stats = coverage_summary[position]
        print(f"{position:3} Position ({len(stats)} stats):")
        for stat in sorted(stats):
            print(f"   ✅ {stat}")
        print()

# Next steps
print("="*80)
print("📋 NEXT STEPS")
print("="*80)
print("""
1. ✅ DONE: Train all models for betting scenarios
2. ✅ DONE: Export models to disk with joblib
3. ✅ DONE: Save feature metadata and preprocessing code

READY FOR PHASE 2:
4. 🔜 Copy saved_models/ folder to Django backend
5. 🔜 Create ML service module (model_loader.py, predictor.py)
6. 🔜 Add prediction API endpoint
7. 🔜 Connect frontend to backend
8. 🔜 Test end-to-end prediction pipeline

Your models are now ready for production deployment! 🚀
""")

print("="*80)
print(f"Notebook execution complete: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)



🎉 PHASE 1 COMPLETE: MODELS EXPORTED SUCCESSFULLY!

📁 Export Directory:
   c:\Users\tonyg\OneDrive\Desktop\University Files\Fall 2025\SWE\Main Project\hedge-your-bets\hedge-your-bets\machine_learning\training_code\..\saved_models

📦 Exported Files (54 total):

   Quantile Models: 51
      • QB_completions_q10.joblib                ( 275.7 KB)
      • QB_completions_q50.joblib                ( 278.6 KB)
      • QB_completions_q90.joblib                ( 258.4 KB)
      • QB_passing_interceptions_q10.joblib      ( 251.7 KB)
      • QB_passing_interceptions_q50.joblib      ( 276.0 KB)
      ... and 46 more model files

   Metadata Files: 2
      • feature_columns.joblib                   (   6.5 KB)
      • model_metadata.joblib                    (   3.2 KB)

   Python Modules: 1
      • inference_preprocessing.py               (   4.5 KB)

   💾 Total Size: 13.56 MB (13884.3 KB)

🏈 MODEL COVERAGE - What Can Be Predicted

QB  Position (5 stats):
   ✅ Completions
   ✅ Passing Interceptions

In [26]:
# Create README for saved_models directory
readme_content = f"""# Hedge Your Bets - Trained ML Models

**Training Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Data Range:** 2020-2024 NFL Season Data
**Total Records:** {len(processed_data):,}

## 📦 Directory Contents

### Model Files (*.joblib)
Quantile regression models trained on NFL player statistics. Each position-stat combination has 3 quantile models:
- `q10` = 10th percentile (pessimistic prediction)
- `q50` = 50th percentile (median/most likely prediction)
- `q90` = 90th percentile (optimistic prediction)

**Naming Convention:** `{{POSITION}}_{{STAT}}_q{{QUANTILE}}.joblib`

Example: `QB_passing_yards_q50.joblib` is the median passing yards model for quarterbacks.

### Metadata Files
- `model_metadata.joblib` - Complete training metadata, stats, and model performance
- `feature_columns.joblib` - Feature lists for each model (CRITICAL for predictions)

### Python Modules
- `inference_preprocessing.py` - Preprocessing utilities for making predictions

## 🏈 Model Coverage

### Quarterback (QB)
{chr(10).join([f'- {stat.replace("_", " ").title()}' for stat in coverage_summary.get('QB', [])])}

### Running Back (RB)
{chr(10).join([f'- {stat.replace("_", " ").title()}' for stat in coverage_summary.get('RB', [])])}

### Wide Receiver (WR)
{chr(10).join([f'- {stat.replace("_", " ").title()}' for stat in coverage_summary.get('WR', [])])}

### Tight End (TE)
{chr(10).join([f'- {stat.replace("_", " ").title()}' for stat in coverage_summary.get('TE', [])])}

## 📊 Model Statistics

- **Algorithm:** LightGBM Quantile Regression
- **Training Method:** Per-position models with position-specific features
- **Features:** 20-27 features per position including:
  - Rolling averages (3-game and 5-game windows)
  - Team context statistics
  - Temporal features (season progression, playoff indicator)
  
## 🚀 Usage in Production

### Loading a Model

```python
import joblib

# Load a model
model = joblib.load('QB_passing_yards_q50.joblib')

# Load feature columns (REQUIRED!)
feature_columns = joblib.load('feature_columns.joblib')
features = feature_columns['QB_passing_yards']

# Make prediction (ensure features match exactly)
prediction = model.predict(your_features_dataframe[features])
```

### Important Notes

1. **Feature Order Matters:** Always use the exact feature list from `feature_columns.joblib`
2. **Rolling Features Required:** Models need player's last 3-5 games to calculate rolling averages
3. **Team Context:** Models expect team-level statistics for the game
4. **Data Preprocessing:** Use `inference_preprocessing.py` to prepare features correctly

## ⚠️ Important Considerations

- Models trained on 2020-2024 data - performance may degrade with future seasons
- Models assume similar stat distributions - major rule changes could affect accuracy
- Always validate that requested position-stat combinations have trained models
- Handle edge cases: rookies (no history), injured players (stale data), etc.

---
*Generated by model_training.ipynb*
*For questions or issues, refer to the training notebook*
"""

readme_path = model_dir / 'README.md'
with open(readme_path, 'w') as f:
    f.write(readme_content)

print(f"\n✅ Created README.md in saved_models directory")
print(f"   Path: {readme_path.absolute()}")



✅ Created README.md in saved_models directory
   Path: c:\Users\tonyg\OneDrive\Desktop\University Files\Fall 2025\SWE\Main Project\hedge-your-bets\hedge-your-bets\machine_learning\training_code\..\saved_models\README.md
